## Splink comparison viewer

`splink_comparison_viewer` is a tool to help you rapidly understand and quality assure the results of a Splink model.  It's a separate package, available from pypi, which can be installed using `pip install spli.nk_comparison_viewer`

In this demo, we use `splink_comparison_viewer` to generate an interactive dashboard.

The model used is the final model produced by the [combining estimates notebook](combining_estimates.ipynb).





## Step 1:  Imports and setup

The following is just boilerplate code that sets up the Spark session and sets some other non-essential configuration options

In [1]:
from utility_functions.demo_utils import get_spark

spark = get_spark()  # See utility_functions/demo_utils.py for how to set up Spark

22/01/11 05:42:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/11 05:42:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/11 05:42:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/11 05:42:39 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


## Load data and Splink model

In [2]:
df = spark.read.parquet("data/fake_1000.parquet")

In [3]:
import json
with open("data/fake_1000_combined.json") as f:
    settings = json.load(f)
    settings = settings["current_settings_dict"]


## Apply pre-trained splink model to data

In [4]:
from splink import Splink

linker = Splink(settings, df, spark)
df_e = linker.manually_apply_fellegi_sunter_weights()

/Users/robinlinacre/anaconda3/lib/python3.8/site-packages/splink/default_settings.py:262: UserWarning: You have not specified any blocking rules, meaning all comparisons between the input dataset(s) will be generated and blocking will not be used.For large input datasets, this will generally be computationally intractable because it will generate comparisons equal to the number of rows squared.
  warnings.warn(
22/01/11 05:42:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Generate dashboard

In [6]:
from splink_comparison_viewer import get_vis_data, render_html_vis

# The '3' parameter is the number of examples to output to the dashboard per comparison vector
edges_data = get_vis_data(df_e, settings, 3)
render_html_vis(edges_data, settings, "splink_comparison_viewer.html", overwrite=True)

In [7]:
from IPython.display import IFrame
IFrame(src="./splink_comparison_viewer.html", width=1000, height=1000)